### Definición del Problema

Sudoku es un puzzle numérico popular para todas las personas y puede ser modelado como un problema de programación lineal, usando variables binarias

La meta (o objetivo principal) del Sudoku es colocar los dígitos del 1 al 9 en un tablero de $9 \times 9$, donde algunas celdas de este tablero ya tienen fijado un dígito. La solución debe seguir las siguientes reglas:
- Los números de 1 al 9 deben estar en cada sub-tablero de $3 \times 3$
- Cada columna debe contener los números del 1 al 9
- Cada fila debe contener los números del 1 al 9

In [9]:
table = [
  [0,0,0, 0,0,0, 0,2,9],
  [0,0,0, 0,7,8, 0,0,5],
  [0,0,0, 1,9,0, 8,3,0],

  [0,6,1, 0,3,9, 2,5,0],
  [0,3,0, 0,0,0, 0,9,0],
  [0,8,9, 4,6,0, 3,7,0],
  
  [0,5,2, 0,1,4, 0,0,0],
  [1,0,0, 3,5,0, 0,0,0],
  [3,7,0, 0,0,0, 0,0,0],
]
G = [ (i+1,j+1,table[i][j]) for i in range(9) for j in range(9) ]

In [10]:
from pulp import *

P = LpProblem("Sudoku-Problem", LpMinimize)

In [11]:
VALS=ROWS=COLS = range(1, 10)
BOXES = [ [(3 *i + k+1, 3*j + l+1) for k in range(3) for l in range(3) ] for i in range(3) for j in range(3) ]

Las variables decisión del problema son:
$$x_{ijk} = \bigg\{ \begin{matrix}
1 & \text{celda }(i,j) = k\\
0 & \text{e.o.c}
\end{matrix}$$

In [12]:
x = LpVariable.dicts("Variable", (ROWS,COLS,VALS), cat="Binary")

La función objetivo se vuelve irrelevante porque cada punto que satisface las restricciones representará una solución del puzzle
$$\begin{matrix}
\min 0^T x_{ijk} & \forall i,j,k
\end{matrix}$$

In [13]:
P += 0*lpSum( [x[i][j][k] for i in ROWS for j in COLS for k in VALS] )


Entonces, tenemos las siguientes restricciones:

Cada fila contiene exactamente un numero entero del 1 al 9
$$\begin{matrix}
\displaystyle\sum_{i=1}^9 x_{ijk} = 1 & \forall j,k
\end{matrix}$$

In [14]:
for k in VALS:
  for j in COLS:
    P += lpSum([x[i][j][k] for i in ROWS]) == 1

Cada columna contiene exactamente un número entero del 1 al 9
$$\begin{matrix}
\displaystyle\sum_{j=1}^9 x_{ijk} = 1 & \forall i,k
\end{matrix}$$

In [15]:
for k in VALS:
  for i in ROWS:
    P += lpSum([x[i][j][k] for j in COLS]) == 1

Cada celda tiene asignado solamente un numero entero del 1 al 9
$$\begin{matrix}
\displaystyle\sum_{k=1}^9 x_{ijk} = 1 & \forall i,j
\end{matrix}$$

In [16]:
for i in ROWS:
  for j in COLS:
    P += lpSum([x[i][j][k] for k in VALS]) == 1

Cada sub-tabla o sub-tablero contiene exactamente todos los números del 1 al 9
$$\begin{matrix}
\displaystyle\sum_{i=3p-2}^{3p} \sum_{j=3q-2}^{3q} x_{ijk} = 1 & \forall k \land \forall p,q \in \{ 1,2,3 \}
\end{matrix}$$

In [17]:
for k in VALS:
  for box in BOXES:
    P += lpSum( [x[i][j][k] for (i,j) in box] ) == 1


In [18]:
for (i,j,k) in G:
  if k > 0: P += x[i][j][k] == 1 

Se debe tener en cuenta que, en la gran mayoría de las tablas, existe una solución única, pero puede existir más de una solución por tabla, e incluso, no tener solución

In [ ]:
P.solve()
LpStatus[P.status]

In [ ]:
for v in P.variables():
  if v.varValue:
    print( f"{v.name} = {v.varValue}" )